In [1]:
dirpath = '../datathon-fme-mango/archive/'
imagepath = dirpath + 'images/images/'

In [20]:
# generate embeddings for each image
import os
import numpy as np
import pandas as pd
import torch
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
import pickle

EMBEDDING_DIM = 1024

# load the model
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])
model.eval()

# load the image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# load the image
def load_image(imagepath):
    image = Image.open(imagepath)
    image = transform(image)
    image = image.unsqueeze(0)
    return image

# generate the embedding
def generate_embedding(imagepath):
    image = load_image(imagepath)
    with torch.no_grad():
        embedding = model(image)
    return embedding

NUM_IM = 1000

# generate the embeddings for all images
imagepaths = []
embeddings = {}
for image in tqdm(os.listdir(imagepath)[:NUM_IM]):
    try:
        imagepaths.append(image)
        embedding = generate_embedding(imagepath + image)
        embeddings[image] = embedding
    except:
        print(f'Error with {image}')

# save the embeddings
embeddings = {k: v.numpy().flatten() for k, v in embeddings.items()}
# save as pickle
with open('embeddings/embeddings_resnet50.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

100%|██████████| 1000/1000 [01:46<00:00,  9.39it/s]
